In [ ]:
# 套件匯入
import pandas as pd
import numpy as np
import os
import re

In [ ]:
# 列出所有期貨報價檔案名稱
filenames = [f for f in os.listdir('../Try crawl/data/') if re.search('^E|^F', f)]
filenames

['ExchangeRate_NTD2USD.csv',
 'Futures_Brent Oil.csv',
 'Futures_Crude Oil.csv',
 'Futures_DalianIronOre.csv',
 'Futures_Gold.csv',
 'Futures_Iron ore fines 62% Fe CFR.csv',
 'Futures_Nickel.csv',
 'Futures_US Midwest Domestic Hot-Rolled Coil Steel.csv']

In [ ]:
# 先觀察一下各個檔案資料長度
for filename in filenames:
    df = pd.read_csv(f'../Try crawl/data/{filename}')
    lendf = len(df)
    print(f'{lendf} is the length of {filename}')

2857 is the length of ExchangeRate_NTD2USD.csv
2968 is the length of Futures_Brent Oil.csv
2985 is the length of Futures_Crude Oil.csv
1879 is the length of Futures_DalianIronOre.csv
2992 is the length of Futures_Gold.csv
2678 is the length of Futures_Iron ore fines 62% Fe CFR.csv
2844 is the length of Futures_Nickel.csv
2931 is the length of Futures_US Midwest Domestic Hot-Rolled Coil Steel.csv


##### 觀察並練習一下

In [ ]:
# 找出標準長度檔案
df_2002 = pd.read_csv('../Try crawl/data/Stock_TW_2002.TW.csv')

In [ ]:
# 拿一個期貨報價來試試看
df_1312 = pd.read_csv('../Try crawl/data/Stock_TW_1312.TW.csv')

In [ ]:
df_1312.head()

NameError: ignored

In [ ]:
df_NTD2USD.info()

NameError: name 'df_NTD2USD' is not defined

In [ ]:
df_NTD2USD.head()

In [ ]:
df_NTD2USD['Date'] = df_NTD2USD['Date'].astype(str).apply(lambda x: '{}-{}-{}'.format(x[:4], x[4:6], x[6:8]))

In [ ]:
df_NTD2USD.info()

In [ ]:
df = pd.merge(df_2002, df_1312, how='left', on='Date', suffixes=('_left',''))

In [ ]:
df.head()

In [ ]:
df = df.drop(['Open', 'High', 'Low', 'Close', 'Volume', 'Dividends', 'Stock Splits'], axis='columns')

In [ ]:
df.info()

In [ ]:
df['Price'] = df['Price'].str.replace(',','').astype(float)
df['Open'] = df['Open'].str.replace(',','').astype(float)
df['High'] = df['High'].str.replace(',','').astype(float)
df['Low'] = df['Low'].str.replace(',','').astype(float)

In [ ]:
df['Vol.'] = df['Vol.'].replace('-', np.nan)
hasM = df['Vol.'].str.contains('M$').replace(np.nan, False)
hasK = df['Vol.'].str.contains('K$').replace(np.nan, False)

In [ ]:
df.isnull().sum()

In [ ]:
df[df.isnull().T.any()]

In [ ]:
# 將 ['Vol.'], ['Change %'] 中字串拿掉，並換成 float 的 type
df['Vol.'] = df['Vol.'].str.replace('K', '').str.replace('M', '').astype(float)
df['Change %'] = df['Change %'].str.replace('%', '').astype(float)

In [ ]:
df['Vol.'] = df['Vol.'].replace(list(df[hasK]['Vol.']), list(df[hasK]['Vol.']*1000))
df['Vol.']

In [ ]:
df['Vol.'] = df['Vol.'].replace(list(df[hasM]['Vol.']), list(df[hasM]['Vol.']*1000000))
df['Vol.']

In [ ]:
# 內插法補空職
df.iloc[:,1:] = df.iloc[:,1:].interpolate()

In [ ]:
df.isnull().any().sum()

### 先整理  'Futures_Brent Oil.csv', 'Futures_Crude Oil.csv', 'Futures_Gold.csv' 

In [1]:
# 建立檔案存檔位置
if os.path.exists('data') is False:
    os.makedirs('data')

NameError: ignored

In [ ]:
filenames = ['Futures_Brent Oil.csv', 'Futures_Crude Oil.csv', 'Futures_Gold.csv']

In [ ]:
# 標準長度的資料讀取
df_2002 = pd.read_csv('../Try crawl/data/Stock_TW_2002.TW.csv')

In [ ]:
for filename in filenames:
    
    # 讀取資料
    dft = pd.read_csv(f'../Try crawl/data/{filename}')
    
    # 兩個 dataframe 合併，並以標準長度 df_2002 的 row 為合併後的 row
    df = pd.merge(df_2002, dft, how='left', on='Date', suffixes=('_left',''))
    
    # 拋棄掉 df_2002 的 columns，保留 dft 的 columns
    df = df.drop(['Open_left', 'High_left', 'Low_left', 'Close', 'Volume', 'Dividends', 'Stock Splits'], axis='columns')
    
    # 整理 ['Price', 'Open', 'High', 'Low'] 為 type = float, learn from Futures_Gold.csv
    if filename == 'Futures_Gold.csv':
        df['Price'] = df['Price'].str.replace(',','').astype(float)
        df['Open'] = df['Open'].str.replace(',','').astype(float)
        df['High'] = df['High'].str.replace(',','').astype(float)
        df['Low'] = df['Low'].str.replace(',','').astype(float)
    
    # 整理 ['Vol.'] 中有 '-' 的換成 np.nan, learn from Futures_Brent Oil.csv
    df['Vol.'] = df['Vol.'].replace('-', np.nan)
    # 整理 ['Vol.'] 中有 'M','K' 的，分別紀錄有 'M' 有 'K' 各自的遮罩 (np.nan 換成 False 才不會有第三個值出現)
    hasM = df['Vol.'].str.contains('M$').replace(np.nan, False) 
    hasK = df['Vol.'].str.contains('K$').replace(np.nan, False)
    
    print(f'After adjust length, the number of NAN in {filename}:')
    print(df.isnull().sum())
    print('')
    
    # 將 ['Vol.'], ['Change %'] 中字串拿掉，並換成 float 的 type
    df['Vol.'] = df['Vol.'].str.replace('K', '').str.replace('M', '').astype(float)
    df['Change %'] = df['Change %'].str.replace('%', '').astype(float)
    
    # 將之前有 'M' 有 'K' 的遮罩換回實際數值, learn from Futures_Brent Oil.csv
    df['Vol.'] = df['Vol.'].replace(list(df[hasK]['Vol.']), list(df[hasK]['Vol.']*1000)) 
    df['Vol.'] = df['Vol.'].replace(list(df[hasM]['Vol.']), list(df[hasM]['Vol.']*1000000))
    
    # 內插法補空職
    df.iloc[:,1:] = df.iloc[:,1:].interpolate()
    
    print('After interpolate:')
    print(df.isnull().sum())
    print('')
    
    # 再次確認 df 都沒有空值且資料長度與 df_2002 相同，再進行存檔
    if df.isnull().any().sum() == 0 and len(df_2002) == len(df):
        
        # 存檔
        df.to_csv(f'data/DateAdj_{filename}', index_label=False )
        print(f'Save file :　DateAdj_{filename}')
        print('-'*100)

### 整理 'ExchangeRate_NTD2USD.csv'

In [ ]:
filename = 'ExchangeRate_NTD2USD.csv'

# 將檔案讀入
dft = pd.read_csv(f'../Try crawl/data/{filename}', index_col = 0)

# 整理 ['Date'] 資料成 20xx-xx-xx
dft['Date'] = dft['Date'].astype(str).apply(lambda x: '{}-{}-{}'.format(x[:4], x[4:6], x[6:8]))

# 兩個 dataframe 合併，並以標準長度 df_2002 的 row 為合併後的 row
df = pd.merge(df_2002, dft, how='left', on='Date', suffixes=('_left',''))

# 拋棄掉 df_2002 的 columns，保留 df_NTD2USD 的 columns
df = df.drop(['Open', 'High', 'Low', 'Close', 'Volume', 'Dividends', 'Stock Splits'], axis='columns')

print(f'After adjust length, the number of NAN in {filename}:')
print(df.isnull().sum())
print('')

# 內插法補空職
df.iloc[:,1:] = df.iloc[:,1:].interpolate()

print('After interpolate:')
print(df.isnull().sum())
print('')
 
# 存檔
df.to_csv(f'data/DateAdj_{filename}', index_label=False )
print(f'Save file :　DateAdj_{filename}')
print('-'*100)

After adjust length, the number of NAN in ExchangeRate_NTD2USD.csv:
Date       0
NTD/USD    9
dtype: int64

After interpolate:
Date       0
NTD/USD    0
dtype: int64

Save file :　DateAdj_ExchangeRate_NTD2USD.csv
----------------------------------------------------------------------------------------------------


In [ ]:
filenames = [f for f in os.listdir('data/')]
filenames

['DateAdj_ExchangeRate_NTD2USD.csv',
 'DateAdj_Futures_Brent Oil.csv',
 'DateAdj_Futures_Crude Oil.csv',
 'DateAdj_Futures_Gold.csv']

In [ ]:
for filename in filenames:
    df = pd.read_csv(f'data/{filename}')
    lendf = len(df)
    print(f'{lendf} is the length of {filename}')

2815 is the length of DateAdj_ExchangeRate_NTD2USD.csv
2815 is the length of DateAdj_Futures_Brent Oil.csv
2815 is the length of DateAdj_Futures_Crude Oil.csv
2815 is the length of DateAdj_Futures_Gold.csv


### 整理 'Futures_DalianIronOre.csv'

In [ ]:
filename = 'Futures_DalianIronOre.csv'

# 將檔案讀入
df_DalianIronOre = pd.read_csv(f'../Try crawl/data/Futures_DalianIronOre.csv')

# 將原本欄位 ['datetime'] 改為 ['Date']
df_DalianIronOre = df_DalianIronOre.rename(columns={'datetime':'Date'})

# 將標準長度調成至 2013-10-18 開始，因此時間才開始有 DalianIronOre
df_2002 = df_2002[df_2002['Date'].between('2013-10-18', '2021-06-30')]

# 兩個 dataframe 合併，並以標準長度 df_2002 的 row 為合併後的 row
df = pd.merge(df_2002, df_DalianIronOre, how='left', on='Date', suffixes=('_left',''))

# 拋棄掉 df_2002 的 columns，保留 df_NTD2USD 的 columns
df = df.drop(['Open', 'High', 'Low', 'Close', 'Volume', 'Dividends', 'Stock Splits'], axis='columns')

print(f'After adjust length, the number of NAN in {filename}:')
print(df.isnull().sum())
print('')

# 內插法補空職
df.iloc[:,1:] = df.iloc[:,1:].interpolate()

print('After interpolate:')
print(df.isnull().sum())
print('')
 
# 存檔
df.to_csv(f'data/DateAdj_{filename}', index_label=False )
print(f'Save file :　DateAdj_{filename}')
print('-'*100)

After adjust length, the number of NAN in Futures_DalianIronOre.csv:
Date       0
open      57
high      57
low       57
close     57
volume    57
dtype: int64

After interpolate:
Date      0
open      0
high      0
low       0
close     0
volume    0
dtype: int64

Save file :　DateAdj_Futures_DalianIronOre.csv
----------------------------------------------------------------------------------------------------


In [ ]:
len(df)